<a href="https://colab.research.google.com/github/rranaaa/Gephi/blob/main/GCN_Accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install torch

In [11]:
pip install torch_geometric

In [22]:
import torch
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid

# Load a sample dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

class SimpleGCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Use data.num_node_features after data is defined
        self.conv1 = GCNConv(data.num_node_features, 16)  # 1st layer: in_dim->4
        self.conv2 = GCNConv(16, dataset.num_classes)     # 2nd layer: 4->num_classes

    def forward(self, data):
      x, edge_index = data.x, data.edge_index
      x = F.relu(self.conv1(x, edge_index))
      x = F.dropout(x, p=0.5, training=self.training)
      x = self.conv2(x, edge_index)  # second layer
      return F.log_softmax(x, dim=1)

model = SimpleGCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Training loop (simplified)
for epoch in range(300):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# Evaluate accuracy on test nodes
model.eval()
pred = model(data).argmax(dim=1)
acc = (pred[data.test_mask] == data.y[data.test_mask]).float().mean()
print(f'GCN Test Accuracy: {acc:.2f}')

GCN Test Accuracy: 0.80
